In [3]:
%%capture --no-stderr
%pip install -U langchain langchain-openai

In [ ]:
# .env setup OR programmatic setup
%pip install openai

import os
import openai
from langsmith import Client

LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT= "https://api.smith.langchain.com"
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY") 
openai.api_key=os.environ["OPENAI_API_KEY"]

# Set tracing in LangGraph
from langchain_core.tracers import ConsoleCallbackHandler
callbacks = [ConsoleCallbackHandler()]  #

In [ ]:

def get_llm():
    return ChatOpenAI(model="gpt-4", temperature=